# Iris classification problem with TensorFlow
This is an example from: https://www.tensorflow.org/get_started/get_started_for_beginners

In [298]:
import argparse
import sys
import os.path
import numpy as np
import pandas as pd
import tensorflow as tf

In [299]:
TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"
DATA_DIR = "./data/"

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

train_file = os.path.abspath(DATA_DIR + TRAIN_URL.split('/')[-1]) # /.../.../iris_training.csv
test_file = os.path.abspath(DATA_DIR + TEST_URL.split('/')[-1]) # /.../../iris_test.csv

## Download and load data
Download the data files if they are not there and save them to current ./data subdirectory.
Load the training set and test set into (train_x, train_y) and (test_x, test_y)

In [300]:
def maybe_download():
    """
    Create a local copy of the training set.
    """
    tf.keras.utils.get_file(train_file, TRAIN_URL)
    tf.keras.utils.get_file(test_file, TEST_URL)

def load_data(y_name='Species'):
    """
    Returns the iris dataset as (train_x, train_y), (test_x, test_y).
    """
    maybe_download()

    train = pd.read_csv(train_file, names=CSV_COLUMN_NAMES, header=0)
    train_x, train_y = train, train.pop(y_name)                                 

    test = pd.read_csv(test_file, names=CSV_COLUMN_NAMES, header=0)
    test_x, test_y = test, test.pop(y_name)

    return (train_x, train_y), (test_x, test_y)

In [301]:
(train_x, train_y), (test_x, test_y) = load_data()

## Examine data

In [302]:
train_x.describe()

,SepalLength,SepalWidth,PetalLength,PetalWidth
count,120.000000,120.000000,120.000000,120.000000
mean,5.845000,3.065000,3.739167,1.196667
std,0.868578,0.427156,1.822100,0.782039
min,4.400000,2.000000,1.000000,0.100000
25%,5.075000,2.800000,1.500000,0.300000
50%,5.800000,3.000000,4.400000,1.300000
75%,6.425000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [303]:
train_y.describe()

count    120.000000
mean       1.000000
std        0.840168
min        0.000000
25%        0.000000
50%        1.000000
75%        2.000000
max        2.000000
Name: Species, dtype: float64

In [304]:
train_x.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [305]:
train_y.head()

0    2
1    1
2    2
3    0
4    0
Name: Species, dtype: int64

In [306]:
test_x.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,5.9,3.0,4.2,1.5
1,6.9,3.1,5.4,2.1
2,5.1,3.3,1.7,0.5
3,6.0,3.4,4.5,1.6
4,5.5,2.5,4.0,1.3


In [307]:
test_y.head()

0    1
1    2
2    0
3    1
4    1
Name: Species, dtype: int64

In [308]:
train_x.keys()

Index(['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth'], dtype='object')

## Describe data
Feature columns describe how to use the input. Create feature columns for all features.

In [309]:
my_feature_columns = []
for key in train_x.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

In [310]:
my_feature_columns

[_NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

## Select the type of model
Build 2 hidden layer DNN with 10, 10 units respectively.

In [311]:
classifier = tf.estimator.DNNClassifier(feature_columns=my_feature_columns,
                                        hidden_units=[10, 10],
                                        n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpy4hlpbfj', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f82039c3940>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Train the model

### Parameters
**batch_size:** The number of examples in a batch. A batch is the set of examples used in on iteration(that is, one gradient update) of model training.
**train_steps:** The steps argument tells train to stop training after the specified number of iterations.

In [312]:
parser = argparse.ArgumentParser()
parser.add_argument('--batch_size', default=100, type=int, help='batch size')
parser.add_argument('--train_steps', default=1000, type=int,
                    help='number of training steps')

_StoreAction(option_strings=['--train_steps'], dest='train_steps', nargs=None, const=None, default=1000, type=<class 'int'>, choices=None, help='number of training steps', metavar=None)

In [313]:
args = parser.parse_args([])
args.batch_size, args.train_steps

(100, 1000)

### train_input_fn

In [314]:
def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    # Return the dataset.
    return dataset

classifier.train(
    input_fn=lambda:train_input_fn(train_x, train_y,
                                   args.batch_size),
    steps=args.train_steps)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpy4hlpbfj/model.ckpt.
INFO:tensorflow:loss = 178.59467, step = 1
INFO:tensorflow:global_step/sec: 607.617
INFO:tensorflow:loss = 10.993369, step = 101 (0.166 sec)
INFO:tensorflow:global_step/sec: 781.758
INFO:tensorflow:loss = 9.263769, step = 201 (0.128 sec)
INFO:tensorflow:global_step/sec: 790.05
INFO:tensorflow:loss = 7.744854, step = 301 (0.127 sec)
INFO:tensorflow:global_step/sec: 827.179
INFO:tensorflow:loss = 9.789001, step = 401 (0.120 sec)
INFO:tensorflow:global_step/sec: 873.473
INFO:tensorflow:loss = 6.0640306, step = 501 (0.115 sec)
INFO:tensorflow:global_step/sec: 858.28
INFO:tensorflow:loss = 5.592351, step = 601 (0.118 sec)
INFO:tensorflow:global_step/sec: 846.068
INFO:tensorflow:loss =

## Evaluate the model

In [315]:
def eval_input_fn(features, labels, batch_size):
    """An input function for evaluation or prediction"""
    features=dict(features)
    if labels is None:
        # No labels, use only features.
        inputs = features
    else:
        inputs = (features, labels)

    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)

    # Return the dataset.
    return dataset


eval_result = classifier.evaluate(
    input_fn=lambda:eval_input_fn(test_x, test_y,
                                            args.batch_size))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-10-16:51:52
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpy4hlpbfj/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-04-10-16:51:53
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.96666664, average_loss = 0.053693816, global_step = 1000, loss = 1.6108145

Test set accuracy: 0.967



## Predicting

In [317]:
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}

predictions = classifier.predict(
    input_fn=lambda:eval_input_fn(predict_x,
                                            labels=None,
                                            batch_size=args.batch_size))

template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')

for pred_dict, expec in zip(predictions, expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print(template.format(SPECIES[class_id], 100 * probability, expec))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpy4hlpbfj/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

Prediction is "Setosa" (99.8%), expected "Setosa"

Prediction is "Versicolor" (99.9%), expected "Versicolor"

Prediction is "Virginica" (97.4%), expected "Virginica"
